<a href="https://colab.research.google.com/github/AshvinVignesh/Final_year/blob/main/Final_year_POS_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.9/386.9 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.0 MB/s eta 0:00:00


In [ ]:
from openai import OpenAI
import asyncio
import json
import os
import pandas as pd
import random
import time

In [ ]:
api_key= ""

client = OpenAI(api_key=api_key)

In [ ]:
response_format = {
    "type": "json_schema",
    "json_schema": {
        "name": "Tamil_Code_Mixed_POS_Tagger",
        "description": "A JSON schema for a POS tagger for Tamil code-mixed reviews",
        "schema": {
            "type": "object",
            "properties": {
                "sentences_with_pos_tags": {
                    "type": "string",
                    "description": "sentences with taged POS tags"
                },

            },
            "required": ["sentences_with_pos_tags","lang","sentiment"],
            "additionalProperties": False
        },
        "strict": True
    }
}

In [ ]:
tamil_code_mixed_prompt = """
You are an AI assistant specialized in identifying Parts of Speech (POS) tags in Tamil code-mixed review data. Your task is to analyze each word in the given text and provide its POS tag .

When given a  review, follow these guidelines:

1. Identify each word in the text.
2. Determine the appropriate POS tag for each word.
3. Use the provided JSON schema to structure your response.
4.Consider the cast names or their abbreviation when classifying sentiment.
Classify the following sentences based on the language used:

POS tags to use:
ADJ (adjective), NOUN, PUNCT (punctuation), AUX (auxiliary), PROPN (proper noun), CONJ (conjunction), VERB, ADV (adverb), DET (determiner), PRON (pronoun), ADP (adposition), NUM (numeral), PART (particle), INTJ (interjection), X (other)

for exmple :
1.இந்த/DET படம்/NOUN super/ADJ அழகாக/ADV இருந்தது/VERB ./PUNCT நடிப்பு/NOUN excellent/ADJ ஆனால்/CONJ கதை/NOUN weak/ADJ ./PUNCT sentiment -> POSITIVE lang -> Code-Switched

Below given details is about "VA VARALAAM VAA" movie Trailer
starring Balaji Murugadoss, Mahana Sanjeevi, Redin Kingsley, Gayatri Rema, Mime Gopi in the lead roles, Directed by SBR - LGR, Music composed by Thenisai Thendral Deva
below is the review
"""

In [ ]:
def get_chat_response(messages):
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        temperature=0.7,
        max_tokens=300,
        response_format = response_format

    )
    return json.loads(response.choices[0].message.content)

In [ ]:
def process_text(text):
    messages = [
        {"role": "system", "content": tamil_code_mixed_prompt},
        {"role": "user", "content": text}
    ]
    response = get_chat_response(messages)
    print(response)
    return response

In [ ]:
import time
import pandas as pd

def main():
    # Load the data
    df = pd.read_csv('VA VARALAM VA - before_label.csv', delimiter=',')
    texts = df['clean_text'].tolist()
    sentences_list = []
    langs_list = []
    sentiment_list = []
    count = 0

    try:
        for text in texts:

            print(f"Processing record {count}")

            # Process the text
            res = process_text(text)
            sentences_with_pos_tags = res['sentences_with_pos_tags']
            lang = res['lang']
            sentiment = res['sentiment']

            # Append results
            sentences_list.append(sentences_with_pos_tags)
            langs_list.append(lang)
            sentiment_list.append(sentiment)

            # Wait for 60 seconds if count is divisible by 100
            if count != 0 and count % 100 == 0:
                print("Waiting for 60 seconds...")
                time.sleep(30)
            if count == 2000:
              break
            count += 1

    except Exception as e:
        print(f"Error occurred at record {count}: {e}")
        save_progress(count, texts, sentences_list, langs_list, sentiment_list)

    # Final save at the end of processing
    save_progress(count, texts, sentences_list, langs_list, sentiment_list)
    print("Processing complete. All data saved successfully.")

def save_progress(count, texts, sentences_list, langs_list, sentiment_list):
    progress_df = pd.DataFrame({
        'original_text': texts[:count],
        'sentences_with_pos_tags': sentences_list,
        'lang': langs_list,
        'sentiment': sentiment_list
    })
    progress_df.to_csv('VA_VARALAM_VA_POS_labeled.csv', index=False)
    print("Progress saved successfully.")


In [ ]:
# def main():
#     # Load the data
#     df = pd.read_csv('VA VARALAM VA - before_label.csv', delimiter=',')

#     # Get total number of rows
#     total_rows = len(df)
#     print("Total rows",total_rows)

#     # Generate 1500 random unique indices
#     random_indices = random.sample(range(total_rows), 600)

#     # Select only those random rows
#     sampled_df = df.iloc[random_indices]
#     texts = sampled_df['clean_text'].tolist()

#     # Initialize lists to store results
#     sentences_list = []
#     langs_list = []
#     sentiment_list = []
#     count = 0
#     print("selected Total rows",len(sampled_df))
#     try:
#         for text in texts:
#             print(f"Processing record {count} of 1000")

#             # Process the text
#             res = process_text(text)
#             sentences_with_pos_tags = res['sentences_with_pos_tags']
#             lang = res['lang']
#             sentiment = res['sentiment']

#             # Append results
#             sentences_list.append(sentences_with_pos_tags)
#             langs_list.append(lang)
#             sentiment_list.append(sentiment)

#             # Wait for 60 seconds if count is divisible by 100
#             if count != 0 and count % 100 == 0:
#                 print("Waiting for 60 seconds...")
#                 time.sleep(60)

#             count += 1

#     except Exception as e:
#         print(f"Error occurred at record {count}: {e}")
#         save_progress(count, texts, sentences_list, langs_list, sentiment_list)

#     # Final save at the end of processing
#     save_progress(count, texts, sentences_list, langs_list, sentiment_list)
#     print("Processing complete. All data saved successfully.")

# def save_progress(count, texts, sentences_list, langs_list, sentiment_list):
#     progress_df = pd.DataFrame({
#         'original_text': texts[:count],
#         'sentences_with_pos_tags': sentences_list,
#         'lang': langs_list,
#         'sentiment': sentiment_list
#     })
#     progress_df.to_csv('GOAT_POS_labeled.csv', index=False)
#     print("Progress saved successfully.")

In [ ]:
if __name__ == "__main__":
    main() # Removed asyncio.run

Processing record 0
{'sentences_with_pos_tags': 'ayyo/INTJ ithuku/PRON legend/PROPN saravanan/PROPN evlo/ADV paravala/ADJ polaye/VERB', 'lang': 'Code-Mixed', 'sentiment': 'NEGATIVE'}
Processing record 1
{'sentences_with_pos_tags': 'பால/PROPN சூப்பர்/ADJ ஸ்டார்/NOUN', 'lang': 'Code-Mixed', 'sentiment': 'POSITIVE'}
Processing record 2
{'sentences_with_pos_tags': 'mm/X high/ADJ budget/NOUN serial/NOUN', 'lang': 'English', 'sentiment': 'NEUTRAL'}
Processing record 3
{'sentences_with_pos_tags': 'totally/ADV looks/VERB ameturish/ADJ ./PUNCT dop/NOUN shameful/ADJ ./PUNCT acting/NOUN worst/ADJ and/CONJ not/PART at/ADP all/DET worth/ADJ to/PART watch/VERB', 'lang': 'English', 'sentiment': 'NEGATIVE'}
Processing record 4
{'sentences_with_pos_tags': '(தல/PROPN பத்தரம்/NOUN ./PUNCT கோடு/PROPN வேர்ட்/X இனி/ADV வருங்கால/NOUN தலயின்/PROPN சொல்லப்போரேன்/VERB ./PUNCT சொல்லப்போரேன்/VERB song/NOUN மன்டைக்குள்ளே/ADV ஓடுது/VERB', 'lang': 'Code-Mixed', 'sentiment': 'NEUTRAL'}
Processing record 5
{'sentences

In [ ]:
{'sentences_with_pos_tags': 'ayyo/INTJ ithuku/PRON legend/PROPN saravanan/PROPN evlo/ADV paravala/ADJ polaye/VERB'}
{'sentences_with_pos_tags': 'பால/PROPN சூப்பர்/ADJ ஸ்டார்/NOUN'}
{'sentences_with_pos_tags': 'mm/X high/ADJ budget/NOUN serial/NOUN'}